In [1]:
import requests
from bs4 import BeautifulSoup

def isInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def checaStr(p,s):
    try:
        p.index(s)
        return True
    except ValueError:
        return False
        


page_ex = requests.get("https://www.mec.ca/en/cat/climb")
soup = BeautifulSoup(page_ex.content, 'html.parser')
#Links referentes a escalada
if (page_ex.status_code) == 200:
    linkExterno = []
    i = 0
    for link in soup.find_all('a'):
        a = link.get('href')
        b = a.split('/')
        if len(b) > 4:
            if b[4] == 'climbing-gear':
                if ("https://www.mec.ca"+ a) not in linkExterno:
                    linkExterno.append("https://www.mec.ca"+ a)

In [ ]:
linkInterno = []
for pagina in linkExterno:
	page_int = requests.get(pagina)
	page_int.status_code
	#Encontrando todas as instâncias de uma tag de uma vez
	soup = BeautifulSoup(page_int.content, 'html.parser')
	for link in soup.find_all('a'):
	        a = link.get('href')
	        b = a.split('/')
	        if len(b) > 4:
	            c = b[3]
	            d = b[4]
	            if len (c) > 1: #Checo se existe algo
	                #se a primeira posicao for um numero e nao tiver # eu adiciono ao vetor
	                if (isInt(c[0]) == True) and (checaStr(d,'#') == False): 
	                    #Se o meu list linkInterno estiver vazio dou append
	                    if ("https://www.mec.ca"+ a) not in linkInterno:
	                        linkInterno.append("https://www.mec.ca"+ a)

In [ ]:
name = []
price = []
infos = []


for pagina in linkInterno:
    if pagina != 'https://www.mec.ca/en/product/5027-227/MEC-E-Gift-Card':
        page_prod = requests.get(pagina)
        soup = BeautifulSoup(page_prod.content, 'html.parser')

        #pego o nome
        name.append(soup.title.string)
        print(pagina)
        # Pego preço
        try:
            price.append(soup.find(itemprop="price").text)
        except AttributeError:
            print ("Checar")
            price.append('Checar')
            



        #Tabela
        tables = soup.findChildren('table')
        #Pegando as infos da tabela de descrição
        tables = soup.findChildren('table')
        #pega a primeira tabela que encontrar
        my_table = tables[0]
        # Acha as childrens da tabela
        rows = my_table.findChildren(['th', 'tr'])
        #pegando as infos da tabela
        infos_prod = []
        for row in rows:
            colunas = row.findChildren('td')
            for coluna in colunas:
                value = coluna.string
                if str(value).strip() != 'None':
                    infos_prod.append(str(value).strip())
        infos.append(infos_prod)


In [ ]:
print(len(name))

In [ ]:
print(len(price))

In [ ]:
print(len(infos))

In [ ]:
import pandas as pd

produtos = pd.DataFrame({
        "Name": name,
        "Price": price,
        "Informations": infos
        })

In [ ]:
writer = pd.ExcelWriter('pandas_simple.xlsx', engine='xlsxwriter')
produtos.to_excel(writer, sheet_name='Sheet1')
writer.save()
